In [1]:
# import tensorflow as tf
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import keras.layers as layers
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler


import numpy as np
import PIL

tf.config.list_physical_devices('GPU')

[]

In [2]:
print('tensorflow:',tf.__version__)
print('keras:',keras.__version__)

tensorflow: 2.6.0
keras: 2.6.0


In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.2)

training_set = train_datagen.flow_from_directory(r'C:\Users\Administrator\Desktop\Dataset\food101',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 101000 images belonging to 101 classes.


In [4]:
len(training_set)

3157

In [5]:
class VGG11(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.conv_layer1 = tf.keras.Sequential([
            layers.Conv2D(filters=64,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer1 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.conv_layer2 = tf.keras.Sequential([
            layers.Conv2D(filters=128,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer2 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        

        self.conv_layer3 = tf.keras.Sequential([
            layers.Conv2D(filters=256,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.conv_layer4 = tf.keras.Sequential([
            layers.Conv2D(filters=256,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer3 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)


        self.conv_layer5 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.conv_layer6 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer4 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)

        self.conv_layer7 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.conv_layer8 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer5 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.flatten = layers.Flatten() 

        self.fc_layer1 = tf.keras.Sequential([
        layers.Dense(4096),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.fc_layer2 = tf.keras.Sequential([
        layers.Dense(4096),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.output_layer = tf.keras.Sequential([
        layers.Dense(101),
        layers.Softmax()
        ])

    def call(self,x):
        x = self.conv_layer1(x)
        x = self.sub_sampling_layer1(x)
        x = self.conv_layer2(x)
        x = self.sub_sampling_layer2(x)
        x = self.conv_layer3(x)
        x = self.conv_layer4(x)
        x = self.sub_sampling_layer3(x)
        x = self.conv_layer5(x)
        x = self.conv_layer6(x)
        x = self.sub_sampling_layer4(x)
        x = self.conv_layer7(x)
        x = self.conv_layer8(x)
        x = self.sub_sampling_layer5(x)
        x = self.flatten(x)
        x = self.fc_layer1(x)
        x = self.fc_layer2(x)
        x = self.output_layer(x)
        
        return x

In [6]:
model = VGG11()
model.build(input_shape=(None,224,224,3))
model.summary()

Model: "vg_g11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 64)      2048      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 112, 112, 128)     74368     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 56, 56, 256)       296192    
_________________________________________________________________
sequential_3 (Sequential)    (None, 56, 56, 256)       591104    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0    

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
stop_callback = EarlyStopping(monitor='val_accuracy',patience=3)

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

lr_callbacks = LearningRateScheduler(scheduler)

In [8]:
sample = np.random.randn(32,224,224,3)

In [9]:
model.predict(sample)

array([[0.00940699, 0.00982758, 0.00998227, ..., 0.00967575, 0.01020838,
        0.00949247],
       [0.0094215 , 0.00983209, 0.0100017 , ..., 0.009766  , 0.01020879,
        0.00955615],
       [0.00945508, 0.00983882, 0.01002563, ..., 0.00972173, 0.01022084,
        0.00951032],
       ...,
       [0.00947587, 0.00983904, 0.00995499, ..., 0.00971266, 0.01025793,
        0.00958909],
       [0.0094539 , 0.00982354, 0.00998302, ..., 0.00971254, 0.01017335,
        0.0095186 ],
       [0.00942012, 0.0098247 , 0.00998904, ..., 0.00971571, 0.01019591,
        0.00954816]], dtype=float32)

---
# `GPU인식이 안돼서 CIFAR-10으로 진행.`  
---

---
### 데이터
---

In [10]:
(X_train,y_train),(X_test,y_test) = cifar10.load_data()

In [11]:
X_train = X_train.astype(float)/255
X_test = X_test.astype(float)/255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [12]:
X_train.shape

(50000, 32, 32, 3)

In [13]:
y_train.shape

(50000, 10)

---
### 모델 정의
---

In [19]:
class LenetLike(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.conv_layer1 = tf.keras.Sequential([
            layers.Conv2D(filters=6,kernel_size=5,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer1 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.conv_layer2 = tf.keras.Sequential([
            layers.Conv2D(filters=16,kernel_size=5,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer2 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.flatten = layers.Flatten() 

        self.fc_layer1 = tf.keras.Sequential([
        layers.Dense(512),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.fc_layer2 = tf.keras.Sequential([
        layers.Dense(512),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.output_layer = tf.keras.Sequential([
        layers.Dense(10),
        layers.Softmax()
        ])

    def call(self,x):
        x = self.conv_layer1(x)
        x = self.sub_sampling_layer1(x)
        x = self.conv_layer2(x)
        x = self.sub_sampling_layer2(x)
        x = self.flatten(x)
        x = self.fc_layer1(x)
        x = self.fc_layer2(x)
        x = self.output_layer(x)
        return x

In [20]:
model = LenetLike()
model.build(input_shape=(None,32,32,3))
model.summary()

Model: "lenet_like_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_16 (Sequential)   (None, 32, 32, 6)         480       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 multiple                  0         
_________________________________________________________________
sequential_17 (Sequential)   (None, 16, 16, 16)        2480      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
sequential_18 (Sequential)   (None, 512)               524800    
_________________________________________________________________
sequential_19 (Sequential)   (None, 512)              

In [21]:
sample = np.random.rand(32,32,32,3)
model.predict(sample).shape

(32, 10)

In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

stop_callback = EarlyStopping(monitor='val_accuracy',patience=5)

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

lr_callbacks = LearningRateScheduler(scheduler)

model.fit(x=X_train,y=y_train,
          batch_size=128,
          epochs=100,
          validation_data=(X_test,y_test),
          callbacks=[stop_callback])

Epoch 1/100
391/391 [==============================] - 27s 68ms/step - loss: 1.5080 - accuracy: 0.4519 - val_loss: 1.3963 - val_accuracy: 0.5052
Epoch 2/100
391/391 [==============================] - 26s 67ms/step - loss: 1.4999 - accuracy: 0.4559 - val_loss: 1.3900 - val_accuracy: 0.5080
Epoch 3/100
279/391 [====================>.........] - ETA: 7s - loss: 1.4924 - accuracy: 0.4574

KeyboardInterrupt: 

In [ ]:
prediction = model.predict(X_test)

In [ ]:
(prediction.argmax(-1) == y_test.argmax(-1)).sum()

6983